In [ ]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# See fonts.txt for instructions on installing fonts
def set_style():
    plt.style.use(["seaborn-white", "seaborn-paper"])
    weight = "normal"  # [ 'normal' | 'bold' | 'heavy' | 'light' | 'ultrabold' | 'ultralight']
    fontsize = 16
    # plt.rcParams['mathtext.fontset'] = "custom"
    # plt.rcParams['mathtext.cal'] = "cmsy10"
    # matplotlib.rc("font", family="sans-serif")
    plt.rc("text", usetex=True)
    params = {
        "font.family": "serif",
        # "font.serif": "Nimbus Roman No9 L",
        "font.serif": "Times",
        # Bitstream Vera Serif, New Century Schoolbook, Century Schoolbook L, Utopia, ITC Bookman, Bookman, Nimbus Roman No9 L, Times New Roman, Times, Palatino, Charter, serif
        "font.weight": weight,
        "mathtext.fontset": "custom",
        "mathtext.cal": "cmsy10",
        "legend.fontsize": fontsize,
        "axes.labelsize": fontsize,
        "axes.labelweight": weight,
        "axes.titlesize": fontsize,
        "axes.titleweight": weight,
        "xtick.labelsize": fontsize,
        "ytick.labelsize": fontsize,
        "figure.titlesize": fontsize,
        "figure.titleweight": weight,
    }
    matplotlib.rcParams.update(params)


def set_fig_size(fig, width, height):
    fig.set_size_inches(width, height)
    plt.tight_layout()


def set_fig_style(fig):
    ax = fig.axes[0]
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    fig.tight_layout()


def set_ax_style(ax):
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)


def save_fig(fig_path, crop=True, dpi=300):
    plt.savefig(fig_path, dpi=dpi, bbox_inches="tight")
    if crop and fig_path.endswith(".pdf"):
        os.system('pdfcrop "{:s}"'.format(fig_path))
        os.system(
            'mv "{:s}-crop.pdf" "{:s}"'.format(fig_path.replace(".pdf", ""), fig_path)
        )


def add_grid_log_plot(ax=None):
    """Add grid lines to a log linear plot."""
    if ax is None:
        ax = plt.gca()
    ax.grid(which='both', linestyle='--', linewidth=0.75, alpha=0.5)


def add_grid_linear_plot():
    """Add grid lines to a linear plot."""
    plt.gca().grid(which='both', linestyle='--', linewidth=1)


set_style()

## Fig 1: Fine-tuning epochs

In [ ]:
nan = np.nan
x_data = [25, 50, 100, 200, 400]
methods = [
    "random",
    "supervised IN1k",
    "MoCo v3",
    "BEiT",
    "MAE",
]
markers = [
    "o",
    "v",
    "s",
    "X",
    "P",
]
colors = plt.get_cmap("Dark2")
plot_fn = plt.semilogx

shuffle_inds = [4, 3, 0, 2, 1]
methods_shuffled = [methods[i] for i in shuffle_inds]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

################ ViT-B ################
data = np.array([
    [ nan, 46.6, 46.4, 47.8, 48.1],  # 25ep
    [45.3, 47.9, 47.8, 49.8, 50.1],  # 50ep
    [48.2, 47.2, 47.9, 49.8, 50.3],  # 100ep
    [48.9,  nan,  nan,  nan,  nan],  # 200ep
    [48.2,  nan,  nan,  nan,  nan],  # 400ep
])

data = data.T[shuffle_inds].T

ax = ax1

for i in range(len(methods)):
    ax.semilogx(
        x_data,
        data[:, i],
        color=colors(shuffle_inds[i]),
        marker=markers[shuffle_inds[i]],
        markersize=9,
        linestyle="-.",
        linewidth=0.5,
    )

ax.set_ylim([45, 51])
ax.set_xticks(x_data)
ax.set_xticklabels(x_data)
ax.set_xlabel("Fine-tuning epochs")
ax.set_ylabel(r"AP$^\textrm{box}$")
# Make font white so that all sizing stays correct
# Crop out extra whitespace in latex
ax.xaxis.label.set_color("white")
ax.tick_params(axis="x", colors="white")
ax.set_title("ViT-B Mask R-CNN", y=0.92, bbox=dict(boxstyle="round", facecolor="white", edgecolor="1.0", pad=0.2))
ax.legend(methods_shuffled, title="Initialization", title_fontsize=13, fontsize=13, frameon=True, loc="lower right")

################ ViT-L ################
data = np.array([
    [ nan, 48.4, 48.7, 52.7, 52.1],
    [46.8, 49.3, 49.3, 53.3, 53.3],
    [49.8, 48.5, 49.4, 52.1, 52.1],
    [50.7,  nan,  nan,  nan,  nan],
    [49.5,  nan,  nan,  nan,  nan],
])


methods_shuffled = [methods[i] for i in shuffle_inds]
data = data.T[shuffle_inds].T

ax = ax2

# reverse as a hack to make MAE appear on top of BEiT
for i in range(len(methods) - 1, -1, -1):
    ax.semilogx(
        x_data,
        data[:, i],
        color=colors(shuffle_inds[i]),
        marker=markers[shuffle_inds[i]],
        markersize=9,
        linestyle="-.",
        linewidth=0.5,
    )

ax.set_xticks(x_data)
ax.set_xticklabels(x_data)
ax.set_yticks(range(46, 54))
ax.set_xlabel("Fine-tuning epochs (log scale)")
ax.set_ylabel(r"AP$^\textrm{box}$")
ax.set_title("ViT-L Mask R-CNN", y=0.92, bbox=dict(boxstyle="round", facecolor="white", edgecolor="1.0", pad=0.2))

################ Finalize and plot ################

add_grid_log_plot(ax1)
add_grid_log_plot(ax2)
set_ax_style(ax1)
set_ax_style(ax2)
set_fig_size(fig, 8, 8)

save_fig("fine_tuning_epochs.pdf")
plt.show()

## Fig 2: pre-training epochs

In [ ]:
x = [100, 200, 400, 800, 1600]
# y = [47.2, 49.5, 51.1, 51.9, 52.1]  # 25-epoch
y = [50.1, 51.5, 52.7, 53.1, 53.3]  # 50-epoch

ax = plt.gca()

ax.semilogx(
    x,
    y,
    color=colors(4),
    marker=markers[4],
    markersize=9,
    linestyle="-.",
    linewidth=0.5,
)
for p_x, p_y in zip(x, y):
    ax.annotate(str(p_y), (p_x, p_y), textcoords="offset points", fontsize=12, xytext=(0,10), ha="center")
ax.set_xticks(x)
ax.set_xticklabels(x)
# ax.set_yticks(range(46, 54))
ax.set_xlabel("Pre-training epochs (log scale)")
ax.set_ylabel(r"AP$^\textrm{box}$")
# ax.set_title("ViT-L Mask R-CNN", y=0.92, bbox=dict(boxstyle="round", facecolor="white", edgecolor="0.3", pad=0.2))

# ax.legend(["MAE"], title_fontsize=16, fontsize=16, frameon=True)

add_grid_log_plot(ax)
set_ax_style(ax)
set_fig_size(plt.gcf(), 8, 3)

save_fig("pre_training_epochs.pdf")

plt.show()

## Fig (not using): Hyperparameter tuning

In [ ]:
def plot_lr_wd_data(data, name, opt_i, opt_j, hide_y=False, hide_x=True):
    norm = matplotlib.colors.Normalize(vmin=43, vmax=49)

    fig, ax = plt.subplots()
    ax.matshow(data, cmap='Purples', norm=norm)

    xlabels = [0.025, 0.050, 0.100, 0.200, 0.400]
    ylabels = ["4.0e-5", "8.0e-5", "1.6e-4", "3.2e-4"]

    ax.xaxis.tick_bottom()
    ax.set_xlabel("weight decay")
    if hide_x:
        # Make font white so that all sizing stays correct
        ax.xaxis.label.set_color("white")
    ax.set_xticks(np.arange(len(xlabels)))
    ax.set_xticklabels(xlabels, minor=False)

    ax.set_yticks(np.arange(len(ylabels)))
    ax.set_yticklabels(ylabels, minor=False)
    ax.set_ylabel("learning rate")
    if hide_y:
        # Make font white so that all sizing stays correct
        # Crop out extra whitespace in latex
        ax.yaxis.label.set_color("white")
        ax.tick_params(axis="y", colors="white")

    for (i, j), z in np.ndenumerate(data):
        if np.isnan(z):
            continue
        if opt_i == i and opt_j == j:
            text = r"\underline{" + "{:0.1f}".format(z) + "}"
        else:
            text = "{:0.1f}".format(z)
        ax.text(
            j,
            i,
            text,
            ha="center",
            va="center",
            fontsize=12,
            usetex=True,
            bbox=dict(boxstyle="round", facecolor="white", edgecolor="0.3", pad=0.2)
        )

    set_fig_size(fig, 4, 4)
    ax.set_title(name)
    save_fig(f"lr_wd_{name.replace(' ', '_')}.pdf")

    plt.show()

In [ ]:
#        0.025  0.050  0.100  0.200  0.400
data_template = np.array([
        [nan,   nan,   nan,   nan,   nan,   ],  # 4e-5
        [nan,   nan,   nan,   nan,   nan,   ],  # 8e-5
        [nan,   nan,   nan,   nan,   nan,   ],  # 1.6e-4
        [nan,   nan,   nan,   nan,   nan,   ],  # 3.2e-4
])


#        0.025  0.050  0.100  0.200  0.400
data_random = np.array([
        [nan,   nan,   nan,   nan,   nan,   ],  # 4e-5
        [nan,   nan,   43.8,  45.3,  nan,   ],  # 8e-5
        [nan,   nan,   45.7,  47.1,  46.6,  ],  # 1.6e-4
        [nan,   nan,   46.9,  47.1,  nan,   ],  # 3.2e-4
])
plot_lr_wd_data(data_random, "random", 2, 3)


#        0.025  0.050  0.100  0.200  0.400
data_supervised = np.array([
        [nan,   nan,   45.2,  nan,   nan,   ],  # 4e-5
        [nan,   46.4,  46.5,  46.5,  nan,   ],  # 8e-5
        [nan,   46.5,  46.4,  46.4,  nan,   ],  # 1.6e-4
        [nan,   46.1,  45.7,  44.2,  nan,   ],  # 3.2e-4
])
plot_lr_wd_data(data_supervised, "supervised IN1k", 1, 2, hide_y=True)


#        0.025  0.050  0.100  0.200  0.400
data_moco = np.array([
        [nan,   nan,   nan,   nan,    nan,   ],  # 4e-5
        [nan,   46.1,  46.1,  46.4,   nan,   ],  # 8e-5
        [nan,   46.4,  46.7,  46.3,   nan,   ],  # 1.6e-4
        [nan,   45.8,  45.8,  44.7,   nan,   ],  # 3.2e-4
])
plot_lr_wd_data(data_moco, "MoCo v3", 2, 2, hide_y=True, hide_x=False)


#        0.025  0.050  0.100  0.200  0.400
data_beit = np.array([
        [nan,   nan,   nan,   nan,   nan,   ],  # 4e-5
        [nan,   46.1,  46.6,  46.4,  nan,   ],  # 8e-5
        [46.8,  47.1,  46.7,  46.0,  nan,   ],  # 1.6e-4
        [nan,   46.3,  45.7,  44.0,  nan,   ],  # 3.2e-4
])
plot_lr_wd_data(data_beit, "BEiT", 2, 1, hide_y=True)


#        0.025  0.050  0.100  0.200  0.400
data_mae = np.array([
        [nan,   46.3,  nan,   nan,   nan,   ],  # 4e-5
        [47.4,  47.8,  47.6,  47.5,   nan,   ],  # 8e-5
        [nan,   47.3,  47.4,  46.5,   nan,   ],  # 1.6e-4
        [nan,   46.3,  45.9,  44.1,   nan,   ],  # 3.2e-4
])
plot_lr_wd_data(data_mae, "MAE token", 1, 1, hide_y=True)

## Fig 3: dp tuning

In [ ]:
def plot_dp_data(data, name, opt, hide_y=False, hide_x=True):
    norm = matplotlib.colors.Normalize(vmin=43, vmax=54)

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    ax1.matshow(data[0, :][np.newaxis, :], cmap='Purples', norm=norm)
    ax2.matshow(data[1, :][np.newaxis, :], cmap='Purples', norm=norm)

    xlabels = [0.0, 0.1, 0.2, 0.3]

    ax1.set_xticks([])
    ax1.set_xticklabels([], minor=False)
    ax2.xaxis.tick_bottom()
    ax2.set_xlabel("drop path rate")
    if hide_x:
        # Make font white so that all sizing stays correct
        ax2.xaxis.label.set_color("white")
    ax2.set_xticks(np.arange(len(xlabels)))
    ax2.set_xticklabels(xlabels, minor=False)

    for ax in (ax1, ax2):
        ax.set_yticks([])
        ax.set_yticklabels([], minor=False)
    ax1.set_ylabel("ViT-B", fontsize=13)
    ax2.set_ylabel("ViT-L", fontsize=13)
    if hide_y:
        # Make font white so that all sizing stays correct
        # Crop out extra whitespace in latex
        for ax in (ax1, ax2):
            ax.yaxis.label.set_color("white")
            ax.tick_params(axis="y", colors="white")

    for r in (0, 1):
        for i, z in enumerate(data[r]):
            if np.isnan(z):
                continue
            if opt[r] == i:
                text = r"\underline{" + "{:0.1f}".format(z) + "}"
            else:
                text = "{:0.1f}".format(z)
            ax = ax1 if r == 0 else ax2
            ax.text(
                i,
                0,
                text,
                ha="center",
                va="center",
                fontsize=12,
                usetex=True,
                bbox=dict(boxstyle="round", facecolor="white", edgecolor="0.3", pad=0.2)
            )

    set_fig_size(fig, 4, 1.9)
    ax1.set_title(name)
    save_fig(f"dp_{name.replace(' ', '_')}.pdf")

    plt.show()

In [ ]:
data_random = np.array([
    [47.1, 47.9, 47.9, 47.5],
    [48.0, 49.1, 49.7, 49.8]
])
plot_dp_data(data_random, "random", (1, 3))


data_supervised = np.array([
    [46.5, 46.6, 46.5, 46.1],
    [47.4, 47.9, 48.0, 48.4]
])
plot_dp_data(data_supervised, "supervised IN1k", (1, 3))


data_moco = np.array([
    [46.7, 46.4, 46.1, 45.6],
    [48.4, 48.6, 48.8, 48.7]
])
plot_dp_data(data_moco, "MoCo v3", (0, 2), hide_x=False)


data_beit = np.array([
    [47.1, 47.0, 46.5, 45.9],
    [51.4, 51.7, 52.3, 51.6]
])
plot_dp_data(data_beit, "BEiT", (0, 2))


data_mae = np.array([
    [47.8, 47.0, 46.4, nan],
    [52.1, 52.9, 52.6, 51.7]
])
plot_dp_data(data_mae, "MAE token", (0, 1))


## TIDE analysis

In [ ]:
# Data exported from devfair
errs_by_init = {'MAE': [1.9510576327989924, 5.829108352724248, 0.9666126532664236, 0.11493284337674936, 3.8514876571176444, 6.240006709239921], 'BEiT': [1.832553846598529, 5.80474220638159, 1.0559608256582464, 0.1126485569129585, 3.8653490363024474, 6.035120236519106], 'MoCo v3': [2.053737304639725, 6.64737918499489, 0.9516454246460739, 0.08993888122245153, 3.6175200444034488, 8.543443906443997], 'Sup': [1.8051209140015914, 6.72691764487368, 0.9650350850485125, 0.10915625049095468, 3.62308589310193, 7.71900591282764], 'Random': [2.0461939041410204, 6.134197547611151, 1.0427584405778703, 0.10337559397176221, 3.5926070687475544, 7.8541103677072215]}
err_types = ['cls', 'loc', 'cls+loc', 'dup', 'bg', 'miss']

In [ ]:
x = np.arange(len(err_types))
w = 0.1

fig, ax = plt.subplots()

ax.bar(x,         errs_by_init["MAE"],     w, label=methods_shuffled[0], color=colors(shuffle_inds[0]))
ax.bar(x + w,     errs_by_init["BEiT"],    w, label=methods_shuffled[1], color=colors(shuffle_inds[1]))
ax.bar(x + 2 * w, errs_by_init["Random"],  w, label=methods_shuffled[2], color=colors(shuffle_inds[2]))
ax.bar(x + 3 * w, errs_by_init["MoCo v3"], w, label=methods_shuffled[3], color=colors(shuffle_inds[3]))
ax.bar(x + 4 * w, errs_by_init["Sup"],     w, label=methods_shuffled[4], color=colors(shuffle_inds[4]))

ax.set_xticks(x + 2 * w)
ax.set_xticklabels(err_types)

ax.set_ylabel(r"$\Delta$AP$^\textrm{box}$@0.5")

ax.legend(title="Initialization", title_fontsize=13, fontsize=13, frameon=True)

fig.tight_layout()
add_grid_log_plot(ax)
set_ax_style(ax)
ax.xaxis.grid(False)

set_fig_size(fig, 8, 3)

save_fig("tide.pdf")

plt.show()